In [2]:
import yfinance as yf
import pandas as pd

# ---- Step 1: Sector EV/EBITDA multiples (tweak as you like) ----
sector_multiples = {
    "Technology": 14.8,
    "Communication Services": 12.7,
    "Consumer Defensive": 11.2,
    "Consumer Cyclical": 12.0,          # <-- Amazon lives here on Yahoo (a.k.a. Consumer Discretionary)
    "Industrials": 10.1,
    "Healthcare": 13.3,
    "Energy": 6.5,
    "Utilities": 9.0,
    "Financial Services": 9.8,
    "Real Estate": 10.2,
    "Basic Materials": 8.7,
}

# Optional: normalize some sector labels you might encounter
sector_aliases = {
    "Consumer Discretionary": "Consumer Cyclical",
}

# ---- Helpers to avoid flaky t.info ----
def _get_info(t):
    # yfinance >=0.2 has get_info(); older versions have .info (lazy-loaded)
    try:
        return t.get_info() or {}
    except Exception:
        try:
            return t.info or {}
        except Exception:
            return {}

def _safe_get_ebitda(t):
    # Prefer financial statements over info blob
    try:
        inc = t.get_income_stmt(freq="annual")
        if isinstance(inc, pd.DataFrame) and "EBITDA" in inc.index:
            s = pd.to_numeric(inc.loc["EBITDA"].dropna(), errors="coerce")
            if not s.empty and pd.notna(s.iloc[0]):
                return float(s.iloc[0])
    except Exception:
        pass
    # Fallback to (legacy) info
    info = _get_info(t)
    return info.get("ebitda")

def _safe_get_sector(t):
    info = _get_info(t)
    sector = info.get("sector") or info.get("sectorDisp") or info.get("sectorKey")
    if sector in sector_aliases:
        sector = sector_aliases[sector]
    return sector

def _safe_get_shares_out(t):
    # fast_info is usually reliable
    try:
        shares = t.fast_info.get("shares_outstanding")
        if shares:
            return int(shares)
    except Exception:
        pass
    # Fallback to info
    info = _get_info(t)
    return info.get("sharesOutstanding")

def _safe_get_cash_debt(t):
    info = _get_info(t)
    debt = info.get("totalDebt", 0) or 0
    cash = info.get("totalCash", 0) or 0
    return float(debt), float(cash)

# ---- Core function ----
def estimate_firm_value_using_multiples(ticker):
    try:
        t = yf.Ticker(ticker)

        ebitda = _safe_get_ebitda(t)
        sector = _safe_get_sector(t)
        shares_out = _safe_get_shares_out(t)
        debt, cash = _safe_get_cash_debt(t)

        # Validate inputs
        if ebitda is None or pd.isna(ebitda) or ebitda == 0:
            return {"Ticker": ticker, "Error": "Missing EBITDA"}
        if not sector:
            return {"Ticker": ticker, "Error": "Missing sector"}
        if sector not in sector_multiples:
            return {"Ticker": ticker, "Sector": sector, "Error": "Sector not in multiples map"}
        if shares_out is None or pd.isna(shares_out) or shares_out == 0:
            return {"Ticker": ticker, "Sector": sector, "Error": "Missing shares outstanding"}

        multiple = sector_multiples[sector]

        # Enterprise value from multiple
        enterprise_value = float(ebitda) * float(multiple)

        # Market cap approximation from EV (EV = MC + Debt - Cash  =>  MC = EV - Debt + Cash)
        market_cap = enterprise_value - float(debt) + float(cash)

        value_per_share = market_cap / float(shares_out)

        return {
            "Ticker": ticker,
            "Sector": sector,
            "EBITDA": round(ebitda / 1e9, 2),  # billions
            "Sector Multiple": multiple,
            "Enterprise Value (B)": round(enterprise_value / 1e9, 2),
            "Estimated Market Cap (B)": round(market_cap / 1e9, 2),
            "Shares Outstanding (B)": round(shares_out / 1e9, 2),
            "Value per Share ($)": round(value_per_share, 2),
            "Error": None,
        }

    except Exception as e:
        return {"Ticker": ticker, "Error": str(e)}

# ---- Step 3: Run on a batch ----
TICKERS = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "PG", "MMM"]
results = [estimate_firm_value_using_multiples(t) for t in TICKERS]
df_multiples = pd.DataFrame(results)

# ---- Step 4: Pretty display ----
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")
display(df_multiples)

,Ticker,Sector,EBITDA,Sector Multiple,Enterprise Value (B),Estimated Market Cap (B),Shares Outstanding (B),Value per Share ($),Error
0,AAPL,Technology,141.70,14.80,"2,097.10","2,050.77",14.84,138.19,None
1,MSFT,Technology,156.53,14.80,"2,316.61","2,299.00",7.43,309.29,None
2,GOOGL,Communication Services,140.84,12.70,"1,788.68","1,842.16",5.82,316.69,None
3,AMZN,Consumer Cyclical,133.83,12.00,"1,605.98","1,539.59",10.66,144.36,None
4,META,Communication Services,94.28,12.70,"1,197.36","1,194.87",2.17,550.89,None
5,PG,Consumer Defensive,24.46,11.20,273.90,246.93,2.34,105.42,None
6,MMM,Industrials,5.79,10.10,58.53,48.96,0.53,91.92,None


In [ ]:
# arima
# facebook profit model
# lsdm